In [ ]:
try:
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys 
except Exception as e:
    print(e)

In [ ]:
data = db.DataBase()

In [ ]:
def getData(ano, formacao, limitePaginas=None):
    
    navegador = webdriver.Chrome()
    navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")

    #Tags de pesquisa
    if formacao != "doutorado":
        navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

        if formacao == "mestrado":
            navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

    navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE) #Buscar por assunto

    #Caixa de texto
    buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
    buscar.send_keys(ano)
    buscar.send_keys(Keys.ENTER)

    time.sleep(2)

    passadas = []

    janelaAtual = navegador.current_window_handle
    passadas.append(navegador.window_handles)

    #Variaveis de controle da paginação
    pagina = 1
    pag = 2

    while(True):
        #Troca de pagina
        try:
            if pagina!=1 and pag<=11 and pagina<=11:
                navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                pag = pag+1 if pag<11 else 3

            if pagina>11 and pag>2:
                navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                pag = pag+1 if pag<13 else 3
        except Exception as e:
            print(e)
            break

        #Limitador de pagínas
        if limitePaginas != None and pagina == limitePaginas:
            break

        time.sleep(2)
        try:
            for i in range(1,11):
                #Selecionar curriculo
                navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

                time.sleep(2)

                #Abrir curriculo
                navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()

                time.sleep(3)

                for janela in navegador.window_handles:
                    if janela not in passadas:
                        navegador.switch_to.window(janela)

                #Pegar dados 
                site = BeautifulSoup(navegador.page_source, 'html.parser')

                artigos = []

                for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                    if "2021" in artigo.text:
                        artigo = artigo.text[10:-4:]
                        text = ""
                        maior = 0
                        for s in artigo.split("."):
                            for ss in s.split(","):
                                if len(ss) > maior:
                                    text = ss
                                    maior = len(ss)
                        artigos.append(text)

                nome = site.findAll('h2', attrs={'class':['nome']})[0].text

                #Inserir no banco de dados
                if artigos != []:
                    data.insertData(nome, formacao, artigos, ano)

                navegador.close()

                time.sleep(1)

                navegador.switch_to.window(janelaAtual)

                #Fechar curriculo
                navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()
        except Exception as e:
            print(e)
        data.connection.commit()
        pagina += 1
    navegador.quit()

In [ ]:
#Variaveis de controle
ano = "2018"
formacao = "mestrado"
limitePaginas = None

getData(ano, formacao, limitePaginas)

In [ ]:
data.selectData(2018) #Dados no banco de 

In [ ]:
data.dellAll(2018)